In [31]:
from openai import OpenAI
from openai import embeddings
from dotenv import load_dotenv

from IPython.display import Markdown, display


from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os
import json

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery, VectorQuery, VectorizedQuery, QueryType

load_dotenv(dotenv_path=".openaidev.env")

True

In [2]:

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]

# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
# model = "gpt-5-mini-2025-08-07"
embedding_model = "text-embedding-3-small"
model = "gpt-5-mini"


# Define the Search Function Tool


In [3]:
def generate_embeddings(text):
    client = OpenAI(
        api_key=api_key,
        organization=org,
        project=project
    )

    embedding = client.embeddings.create(
        model=embedding_model,
        input=text
    )
    embedding = embedding.data[0].embedding
    return embedding

In [4]:
# define a search function

def search_function(query_text, top_k=5, vector_only=False):
    vector = generate_embeddings(query_text)
    vector_query = VectorizedQuery(
        vector=vector,
        k_nearest_neighbors=top_k,
        fields="content_vector",
        exhaustive=True,
    )

    search_client = SearchClient(
        endpoint,
        index_name,
        credential=credential)

    search_text = None if vector_only else query_text
    results = search_client.search(
        search_text=search_text,
        vector_queries=[vector_query],
        select=["chunk_id", "file_name", "base_name", "content", "content_vector"],
        top=top_k,
        include_total_count=True,
        # query_type=QueryType.FULL,
        semantic_configuration_name="my-semantic-config",
    )
    return results

In [5]:
def get_search_intent(user_query):

    client = OpenAI(
        api_key=api_key,
        organization=org,
        project=project
    )

    #extract user search intent
    SEARCH_INTENT_PROMPT = """
    You are asked to read the user query, understand it, and then turn it into a search query.
    The search query should be sent to the search engine to find the relevant information.
    the search query should be similar to what a user would type into a search engine, and similar in meaning to the provided user query.
    the user query is: '{user_query}'
    return only the search query text. """

    # print(SEARCH_INTENT_PROMPT.format(user_query=user_query))

    response = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": SEARCH_INTENT_PROMPT.format(user_query=user_query)},
        ]
    )

    search_query = response.output_text

    # print("search query intent: " , search_query)

    return search_query

In [ ]:
get_search_results_function_definition = {

    "type": "function",
    "name": "get_search_results",
    "description": """
    Use this tool to retrieve relevant information and additional context from a knowledge base of HR-related documents (including health insurance plans, coverage, benefits, and more).
    Whenever you need to answer a user question that requires facts, details, or supporting evidence from these documents, call this tool with the user's query.
    The tool returns a list of sources and their content that can be used to provide accurate, grounded answers.
    Always invoke this tool when you need to look up or verify information from the knowledge base.
    """,
    "parameters": {
        "type": "object",
        "properties": {
            "user_query": {
                "type": "string",
                "description": "The user's question or query.",
            }
        },
        "required": ["user_query"],
        "additionalProperties": False
    },
    "strict": True
}

tools = [get_search_results_function_definition]

In [105]:
def get_search_results(user_query: str) -> str:
    # Set query parameters for grounding the conversation on your search index
    search_type="text"
    use_semantic_reranker=True
    sources_to_include=5

    search_query = get_search_intent(user_query=user_query)

    search_results = search_function(query_text=search_query, top_k=sources_to_include, vector_only=False)

    list_of_sources = [
        f'\nSOURCE NO {index + 1}. {result["base_name"]}({result["chunk_id"]}): {result["content"]} )'
        for index, result in enumerate(search_results)
        ]

    joined_sources = "\n".join(list_of_sources)

    return joined_sources


In [106]:
print(tools)

[{'type': 'function', 'name': 'get_search_results', 'description': "\n    Use this tool to retrieve relevant information and additional context from a knowledge base of HR-related documents (including health insurance plans, coverage, benefits, and more). \n    Whenever you need to answer a user question that requires facts, details, or supporting evidence from these documents, call this tool with the user's query.\n    The tool returns a list of sources and their content that can be used to provide accurate, grounded answers.\n    Always invoke this tool when you need to look up or verify information from the knowledge base.\n    ", 'parameters': {'type': 'object', 'properties': {'user_query': {'type': 'string', 'description': "The user's question or query."}}, 'required': ['user_query'], 'additionalProperties': False}, 'strict': True}]


In [107]:
display(Markdown(get_search_results("eye test")))


SOURCE NO 1. Northwind_Health_Plus_Benefits_Details.md(Northwind_Health_Plus_Benefits_Details_0073): ### Psychological and Neuropsychological Testing COVERED SERVICES - Psychological and Neuropsychological Testing  
Northwind Health Plus recognizes the importance of mental health care and offers
psychological and neuropsychological testing as a covered service. In this section, we will
cover what these tests are, what they cover, and what the exceptions are.  
What are Psychological and Neuropsychological Tests?  
Psychological and neuropsychological tests are tests used to diagnose and treat mental
health conditions and disorders. Psychological tests are used to assess personality,
behavior, and emotions, while neuropsychological tests are used to diagnose and treat
neurological disorders.  
What do these Tests Cover?  
Psychological and neuropsychological testing can cover a wide range of topics, including
memory, concentration, and attention; language and communication; motor skills; problem-
solving; and executive functioning. Tests may also assess mood and behavior and can help
to diagnose conditions such as anxiety, depression, and bipolar disorder. )

SOURCE NO 2. Northwind_Health_Plus_Benefits_Details.md(Northwind_Health_Plus_Benefits_Details_0075): ## Tips for Employees  
If you think you may need psychological or neuropsychological testing, be sure to discuss
this with your primary care physician or mental health provider. These tests can be
expensive and time-consuming, so it's important to make sure that any testing you have is
necessary and covered under your health plan. In addition, you should research providers
who offer these services and make sure they are in-network with Northwind Health Plus so
that you can receive the maximum benefit. Finally, make sure to keep track of all of your
medical records and any tests you have so that you can provide this information to your
providers if necessary.  
By understanding what psychological and neuropsychological tests are, what they cover,
and any exceptions to coverage, you can make sure that you are getting the most out of your  
<!-- PageBreak -->  
Northwind Health Plus plan. By taking the time to research providers in-network with
Northwind Health Plus and keeping track of your medical records and tests, you can make
sure you are receiving the care and coverage you need. )

SOURCE NO 3. Northwind_Standard_Benefits_Details.md(Northwind_Standard_Benefits_Details_0074): associated with psychological or neuropsychological testing that is done for non-medical
reasons such as pre-employment screening or for legal purposes.  
In order to ensure that your psychological or neuropsychological testing services are
covered under the Northwind Standard plan, it is important to review your plan's Summary
of Benefits and Coverage (SBC) document. This document will provide you with a full list of
covered services, as well as any exceptions that may apply.  
Overall, the Northwind Standard plan provides coverage for psychological and
neuropsychological testing services when they are prescribed or ordered by a health care
professional and when they are provided by an in-network provider. It is important to be
aware of any exceptions that may apply, as well as to select a highly qualified in-network
provider for your tests in order to ensure that your services are covered in full.  
<!-- PageBreak --> )

SOURCE NO 4. Northwind_Health_Plus_Benefits_Details.md(Northwind_Health_Plus_Benefits_Details_0074): ## Exceptions  
There are some exceptions to the coverage for psychological and neuropsychological tests.
These include tests for intelligence, achievement, and aptitude. In addition, tests that are
intended to evaluate an individual's ability to perform specific job functions are also not
covered under Northwind Health Plus. )

SOURCE NO 5. Northwind_Health_Plus_Benefits_Details.md(Northwind_Health_Plus_Benefits_Details_0028): ### Tips  
· Talk to your doctor about all of your allergy symptoms, so they can determine the best
course of testing and treatment.  
. Ask your doctor about the cost of any allergy medications that they prescribe.  
. Be sure to fill any prescriptions at a pharmacy that is in-network, so you can receive the
lowest cost for your medications.  
· If you are considering an alternative treatment for your allergies, be sure to discuss it with
your doctor first.  
· Keep your receipts for any out-of-pocket expenses related to your allergy testing and
treatment, so you can be reimbursed for your expenses.  
At Contoso, we are committed to providing our employees with the best coverage for their
healthcare needs. With Northwind Health Plus, you can be sure that you are getting the best
coverage for your allergy testing and treatment. )

In [108]:
 # This prompt provides instructions to the model
 GROUNDED_PROMPT="""
 # PURPOSE:
 You are a friendly assistant that answer questions from company's HR policy and information documents.
 Answer the query using only the sources provided below in a friendly and concise bulleted manner.
 Answer ONLY with the facts listed in the list of sources below.
 If there isn't enough information below, say you don't know.
 Do not generate answers that don't use the sources below.

 # CITATION:
 Always provide citations inline from sources provided, and also include a list of sources as a foot note in bullet points format. example list of sources:
 Only include sources that were used to generate the answer. example list of sources:
 ### Sources:
 - SOURCE NO 1. filename1.pdf(chunk id number))
 - SOURCE NO 2. filename2.pdf(3)
 - SOURCE NO 3. filename3.pdf(5)
 Sources:\n{sources}

 Semantic Answer:\n{semantic_response}
 """

# Make a call with tools

In [109]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

In [136]:

input_list = [
    # {
    #     "role": "system",
    #     "content": """
    #     You are an expert assistant. For every user question, think carefully and reason step by step before responding.
    #     Always consider what information is needed to provide a complete and accurate answer.
    #     If a tool is available that can help you retrieve, verify, or supplement information, use it in the most effective way possible.
    #     When in doubt, prefer using tools to gather supporting evidence or additional context before answering.
    #     Your goal is to provide thorough, well-grounded, and helpful responses by leveraging all available resources.

    #     """
    # },
    {
        "role": "user",
        # "content": "Is dental included in the Health plans?"
        # "content": "Hello!"
        # "content": "What is the weather in Paris?"
        "content": "Describe the performance review process."
    }
]


response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
    instructions="""
        You are an expert assistant. For every user question, think carefully and reason step by step before responding.
        Always consider what information is needed to provide a complete and accurate answer.
        If a tool is available that can help you retrieve, verify, or supplement information, use it in the most effective way possible.
        When in doubt, prefer using tools to gather supporting evidence or additional context before answering.
        Your goal is to provide thorough, well-grounded, and helpful responses by leveraging all available resources.

        """,
    tool_choice="auto"
)

In [137]:
print(response.model_dump_json(indent=2))

{
  "id": "resp_68d494433e94819c84462f3e14e498850abe51d28273e85e",
  "created_at": 1758762051.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "\n        You are an expert assistant. For every user question, think carefully and reason step by step before responding.\n        Always consider what information is needed to provide a complete and accurate answer.\n        If a tool is available that can help you retrieve, verify, or supplement information, use it in the most effective way possible.\n        When in doubt, prefer using tools to gather supporting evidence or additional context before answering.\n        Your goal is to provide thorough, well-grounded, and helpful responses by leveraging all available resources.\n\n        ",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_68d494445e78819cbe93742d81c6cd360abe51d28273e85e",
      "summary": [],
      "type": "reasoning",
      "content": null,
    

In [138]:
# inspect response for function calls
final_response = response.output_text
for tool_call in response.output:
    if tool_call.type != "function_call":
        continue

    function_name = tool_call.name
    arguments = json.loads(tool_call.arguments)
    tool_call_id = tool_call.call_id

    function_call_results = locals()[function_name](**arguments)

    function_call_result_message = {
        "type": "function_call_output",
        "call_id": tool_call_id,
        "output": json.dumps(function_call_results)
    }

    input_list += response.output
    input_list += [{"role": "system", "content": GROUNDED_PROMPT.format(sources=function_call_results, semantic_response="")},]
    input_list += [function_call_result_message]


    response_ag = client.responses.create(
        model=model,
        tools=tools,
        input=input_list,
        instructions="use available tools if there is a need to retrieve knowledge for the answer.",
        tool_choice="auto"
    )
    final_response = response_ag.output_text



In [139]:
display(Markdown(final_response))

- Reviews are conducted annually and are an important part of career development; supervisors discuss your performance over the past year and provide feedback on areas for improvement (SOURCE NO 1).
- The review is also an opportunity to discuss goals and objectives for the upcoming year (SOURCE NO 1).
- Performance reviews are a two-way dialogue; employees are encouraged to be honest and open about successes and challenges (SOURCE NO 1).
- Managers aim to provide positive, constructive feedback to help employees develop and grow in their roles (SOURCE NO 1).
- Employees receive a written summary of the review that includes a performance rating, feedback, and goals/objectives for the coming year (SOURCE NO 1).
- The company acknowledges reviews can be stressful and is committed to supporting employees; staff are encouraged to reach out to managers with questions or concerns (SOURCE NO 1).

### Sources:
- SOURCE NO 1. employee_handbook.md(employee_handbook_0004)

In [140]:
print(response_ag.model_dump_json(indent=2))

{
  "id": "resp_68d4944c1874819ca2dc7fe3b4fbb00b0abe51d28273e85e",
  "created_at": 1758762060.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "use available tools if there is a need to retrieve knowledge for the answer.",
  "metadata": {},
  "model": "gpt-5-mini-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_68d4944c96a8819cb691546e8e93cb950abe51d28273e85e",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "id": "msg_68d49454af44819ca0c6cbe96aab397e0abe51d28273e85e",
      "content": [
        {
          "annotations": [],
          "text": "- Reviews are conducted annually and are an important part of career development; supervisors discuss your performance over the past year and provide feedback on areas for improvement (SOURCE NO 1).\n- The review is also an opportunity to discuss goals and objectives for the upcoming year (SOURCE NO 1).\n- Pe